In [ ]:
# --- 1. IMPORTACIÓN Y CONFIGURACIÓN ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

print("📚 Librerías importadas correctamente.")

In [ ]:
# --- 2. CARGA Y PREPROCESAMIENTO DE DATOS ---
try:
    # Carga de datos desde el archivo JSON local
    df_original = pd.read_json('TelecomX_Data.json')
    print("✅ Datos cargados exitosamente desde 'TelecomX_Data.json'.")

    # --- Transformación y Limpieza (similar al script anterior) ---
    df = df_original.copy()
    for columna in ['customer', 'phone', 'internet', 'account']:
        df_anidado = pd.json_normalize(df[columna])
        df = pd.concat([df.drop(columna, axis=1), df_anidado], axis=1)

    # Renombramos las columnas para mayor claridad
    df.rename(columns={
        'customerID': 'id_cliente', 'Churn': 'abandono', 'gender': 'genero',
        'SeniorCitizen': 'adulto_mayor', 'Partner': 'tiene_pareja', 'Dependents': 'tiene_dependientes',
        'tenure': 'meses_contrato', 'PhoneService': 'servicio_telefonico', 'MultipleLines': 'multiples_lineas',
        'InternetService': 'servicio_internet', 'OnlineSecurity': 'seguridad_online',
        'OnlineBackup': 'backup_online', 'DeviceProtection': 'proteccion_dispositivo',
        'TechSupport': 'soporte_tecnico', 'StreamingTV': 'streaming_tv', 'StreamingMovies': 'streaming_peliculas',
        'Contract': 'tipo_contrato', 'PaperlessBilling': 'facturacion_electronica',
        'PaymentMethod': 'metodo_pago', 'Charges.Monthly': 'gasto_mensual', 'Charges.Total': 'gasto_total'
    }, inplace=True)

    # Limpieza de datos
    df['abandono'].replace('', np.nan, inplace=True)
    df['gasto_total'] = pd.to_numeric(df['gasto_total'], errors='coerce')
    df.dropna(inplace=True)

    # Eliminamos el ID de cliente, ya que no es útil para el modelo
    df.drop('id_cliente', axis=1, inplace=True)

    print("🔧 Preprocesamiento y limpieza de datos completados.")

except FileNotFoundError:
    print("❌ Error: Asegúrate de que el archivo 'TelecomX_Data.json' esté en la misma carpeta.")

In [ ]:
# --- 3. CODIFICACIÓN DE VARIABLES CATEGÓRICAS ---
# El modelo necesita que todas las entradas sean numéricas.
# Usaremos LabelEncoder para convertir las columnas de texto a números.
df_codificado = df.copy()
for columna in df_codificado.select_dtypes(include=['object']).columns:
    if columna != 'abandono': # No codificamos la variable objetivo aún
        df_codificado[columna] = LabelEncoder().fit_transform(df_codificado[columna])

# Convertimos la variable objetivo 'abandono' a 0 y 1
df_codificado['abandono'] = df_codificado['abandono'].apply(lambda x: 1 if x == 'Yes' else 0)

print("🔄 Variables categóricas codificadas a formato numérico.")
print("--- Primeros 5 registros de los datos listos para el modelo: ---")
print(df_codificado.head())
print("\n" + "="*50 + "\n")

In [ ]:
# --- 4. PREPARACIÓN PARA EL ENTRENAMIENTO DEL MODELO ---
# Separamos las características (X) de la variable objetivo (y)
X = df_codificado.drop('abandono', axis=1)
y = df_codificado['abandono']

# Dividimos los datos en conjuntos de entrenamiento y prueba (80% para entrenar, 20% para probar)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42, # 'random_state' asegura que la división sea siempre la misma
    stratify=y # 'stratify' mantiene la proporción de 'abandono' en ambos conjuntos
)

print(f"📊 Datos divididos: {len(X_train)} para entrenamiento y {len(X_test)} para prueba.")
print("\n" + "="*50 + "\n")

In [ ]:
# --- 5. ENTRENAMIENTO DEL MODELO DE MACHINE LEARNING ---
# Usaremos un clasificador de Bosque Aleatorio (Random Forest), que es potente y versátil.
print("🧠 Entrenando el modelo de clasificación (Random Forest)...")
modelo = RandomForestClassifier(
    n_estimators=100, # Número de "árboles" en el bosque
    random_state=42,
    max_depth=10 # Profundidad máxima para evitar sobreajuste
)
modelo.fit(X_train, y_train)
print("✅ Modelo entrenado con éxito.")
print("\n" + "="*50 + "\n")

In [ ]:
# --- 6. EVALUACIÓN DEL RENDIMIENTO DEL MODELO ---
# Realizamos predicciones sobre el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calculamos la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Precisión del modelo: {accuracy:.2%}")
print("\n--- Reporte de Clasificación Detallado: ---")
print(classification_report(y_test, y_pred, target_names=['No Abandona', 'Abandona']))

# --- Visualización de la Matriz de Confusión ---
print("📈 Visualizando la Matriz de Confusión...")
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Abandona', 'Abandona'])

disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.show()

print("\n🎉 Análisis completado.")